In [1]:
import pyodbc 
import re
import pandas as pd
import sqlalchemy as sa
import pydrill as pyd
from pyspark.sql import SparkSession

## Create sqlalchemy connection

In [2]:
# Install sqlalchemy drill dialect using the following command:
# pip install sqlalchemy-drill
sa_conn = 'drill+sadrill://drill1:8047/dfs?use_ssl=False'
engine = sa.create_engine(sa_conn)

## Create ODBC connection

In [3]:
MY_DSN = """
Driver                     = /opt/mapr/drill/lib/64/libdrillodbc_sb64.so
ConnectionType             = Zookeeper
ZKQuorum                   = zoo:2181
ZKClusterID                = /drill/drillbits1
AuthenticationType         = No Authentication
TimestampTZDisplayTimezone = utc
ExcludedSchemas            = sys,INFORMATION_SCHEMA
Root                       = ""
"""


In [4]:
MY_DSN = ";".join(
    [re.sub(r'(\t+|\s+)=\s+', '=', i) for i in MY_DSN.split('\n') if i != '']
)
MY_DSN
conn = pyodbc.connect(MY_DSN, autocommit=True)
cursor = conn.cursor()

## Query using spark

In [5]:
spark = SparkSession.builder.appName(__name__).master("local").getOrCreate()

In [6]:
hdfs_file_path = '''test.csv'''
query = f'select * from dfs.root."{hdfs_file_path}" limit 2'

In [7]:
df = spark.read.format("jdbc").option("url", "jdbc:drill:zk=zoo;").option("driver","org.apache.drill.jdbc.Driver").option("query", query).load()


In [8]:
df.show(5)

+---------+--------+
|firstname|lastname|
+---------+--------+
|     John|     Doe|
+---------+--------+



## Query using pandas

In [9]:
pdf = pd.read_sql(query, conn)

In [10]:
pdf

,firstname,lastname
0,John,Doe


## Query using pyodbc cursor object

In [11]:
cursor.execute(query)

In [12]:
data = cursor.fetchall()

In [13]:
results = list()
columns = [column[0] for column in cursor.description]

In [14]:
for row in data:
    results.append(dict(zip(columns, row)))

In [15]:
results

[{'firstname': 'John', 'lastname': 'Doe'}]